1- Veriyi kafkaya atmak.
2- spark streaming ile kafkadan veriyi dinleme
3- gün dönüşümlerini yapmak. ******
4- null veri kontrolü yapmak. 
5- latitude ve longitude kullanıp mesafa ,ortalama hız, hesaplamaları 
6- test içinde future engineering ve ön işleme yapılması
7- kafkada topiclere bölme (tahmin edilen trip duration) 
8- elasticsearch

Bağımsız değişkenler 
1- vendor_id
2- dayofweek
3- month
4 mesafe
5 ortalamahız
Bağımlı değişken tripduration

In [1]:
import findspark

In [2]:
# /opt/manual/spark: this is SPARK_HOME path
findspark.init("/opt/manual/spark")

In [3]:
from pyspark.sql import SparkSession,functions as F 

In [4]:
spark = SparkSession.builder \
.appName("Read Data From File") \
.master("local[2]") \
.getOrCreate()

2022-02-06 21:16:01,153 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
df_train = spark.read.option("header", True) \
    .option("inferSchema", True) \
    .csv("file:///home/train/datasets/nyc_taxi.csv")
df_train.limit(5).toPandas()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [6]:
df_train.printSchema()

root
 |-- id: string (nullable = true)
 |-- vendor_id: integer (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- trip_duration: integer (nullable = true)



In [7]:
df_train.count()

1458644

In [8]:
df_train_last = df_train.withColumn("pickup_datetime", F.to_timestamp(F.col("pickup_datetime"),"yyyy-MM-dd HH:mm:ss")) \
        .withColumn("dropoff_datetime", F.to_timestamp(F.col("dropoff_datetime"),"yyyy-MM-dd HH:mm:ss"))

In [9]:
df_train_last.printSchema()

root
 |-- id: string (nullable = true)
 |-- vendor_id: integer (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- trip_duration: integer (nullable = true)



In [10]:
df_train_last.limit(5).toPandas()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [13]:
df_train.select(['id']).distinct().count()

1458644

In [12]:
df_train_last.agg({"pickup_datetime": "max"}).collect()[0] 

Row(max(pickup_datetime)=datetime.datetime(2016, 6, 30, 23, 59, 39))

In [13]:
df_train_last.agg({"pickup_datetime": "min"}).collect()[0] 

Row(min(pickup_datetime)=datetime.datetime(2016, 1, 1, 0, 0, 17))

In [16]:
jdbcUrl = "jdbc:postgresql://localhost/traindb?user=train&password=Ankara06"

In [17]:
df_train_last.write.jdbc(url=jdbcUrl,
              table="nyc_taxi", 
              mode="overwrite",
              properties={"driver": 'org.postgresql.Driver'})

In [18]:
# check the data in Postgre
(spark.read
.format("jdbc")
.option("url", jdbcUrl)
.option("driver", 'org.postgresql.Driver')
.option("query","select * from nyc_taxi")
.load()).limit(5).toPandas()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id1090265,1,2016-02-09 21:30:46,2016-02-09 21:34:40,1,-74.003197,40.727230,-73.998123,40.735619,N,234
1,id0429543,2,2016-05-26 18:12:10,2016-05-26 18:22:10,1,-73.972610,40.747501,-73.962303,40.759430,N,600
2,id0588876,1,2016-03-12 23:59:16,2016-03-13 00:24:17,1,-74.006866,40.706001,-73.918991,40.741142,N,1501
3,id1046728,1,2016-04-16 03:18:55,2016-04-16 03:37:39,3,-74.008781,40.716190,-73.961594,40.679806,N,1124
4,id2457924,1,2016-03-02 22:53:19,2016-03-02 22:57:02,1,-73.982727,40.772102,-73.985550,40.763416,N,223


In [14]:
dftrain_last_1 = df_train_last \
       .withColumn("pickup_year",
                    F.year(F.to_date(F.col("pickup_datetime")))) \
       .withColumn("pickup_month",
                    F.month(F.to_date(F.col("pickup_datetime")))) \
       .withColumn("pickup_dayofweek",
                    F.dayofweek(F.to_date(F.col("pickup_datetime")))) \
       .withColumn("pickup_hour",
                    F.hour(F.col("pickup_datetime"))) \
       .withColumn("dropoff_year",
                    F.year(F.to_date(F.col("dropoff_datetime")))) \
       .withColumn("dropoff_month",
                    F.month(F.to_date(F.col("dropoff_datetime")))) \
       .withColumn("dropoff_dayofweek",
                    F.dayofweek(F.to_date(F.col("dropoff_datetime")))) \
       .withColumn("dropoff_hour",
                    F.hour(F.col("dropoff_datetime"))) 

In [15]:
dftrain_last_1.limit(5).toPandas()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_year,pickup_month,pickup_dayofweek,pickup_hour,dropoff_year,dropoff_month,dropoff_dayofweek,dropoff_hour
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,2016,3,2,17,2016,3,2,17
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,2016,6,1,0,2016,6,1,0
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,2016,1,3,11,2016,1,3,12
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,2016,4,4,19,2016,4,4,19
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,2016,3,7,13,2016,3,7,13


In [16]:
from pyspark.sql.types import StringType

def switch_tr_day(day_index):
    my_dict = {
        1: 'Pazar',
        2: 'Pazartesi',
        3: 'Salı',
        4: 'Çarşamba',
        5: 'Perşembe',
        6: 'Cuma',
        7: 'Cumartesi'
    }
    
    return my_dict.get(day_index)

In [17]:
switch_day_func = F.udf(lambda z: switch_tr_day(z), StringType())
spark.udf.register("switch_day_func", switch_day_func)

<function __main__.<lambda>(z)>

In [18]:
df7 = dftrain_last_1 \
.withColumn("pickupDayofWeek_TR", switch_day_func(F.col("pickup_dayofweek"))) \
.withColumn("dropoffDayofWeek_TR", switch_day_func(F.col("dropoff_dayofweek")))

df7.limit(5).toPandas()

Traceback (most recent call last):                                  (0 + 1) / 1]
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,pickup_year,pickup_month,pickup_dayofweek,pickup_hour,dropoff_year,dropoff_month,dropoff_dayofweek,dropoff_hour,pickupDayofWeek_TR,dropoffDayofWeek_TR
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,...,2016,3,2,17,2016,3,2,17,Pazartesi,Pazartesi
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,...,2016,6,1,0,2016,6,1,0,Pazar,Pazar
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,...,2016,1,3,11,2016,1,3,12,Salı,Salı
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,...,2016,4,4,19,2016,4,4,19,Çarşamba,Çarşamba
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,...,2016,3,7,13,2016,3,7,13,Cumartesi,Cumartesi


In [19]:


def switch_month_day(month_index):
    my_dict = {
        1: 'Ocak',
        2: 'Subat',
        3: 'Mart',
        4: 'Nisan',
        5: 'Mayis',
        6: 'Haziran',
        7: 'Temmuz',
        8: 'Agustos',
        9: 'Eylul',
        10: 'Ekim',
        11: 'Kasim',
        12: 'Aralik'
    }
    
    return my_dict.get(month_index)

In [20]:
switch_month = F.udf(lambda z: switch_month_day(z), StringType())
spark.udf.register("switch_month", switch_month)

<function __main__.<lambda>(z)>

In [21]:
df8 = df7 \
.withColumn("dropoffMonth_TR", switch_month(F.col("dropoff_month"))) \
.withColumn("pickupMonth_TR", switch_month(F.col("pickup_month")))

df8.limit(5).toPandas()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,pickup_dayofweek,pickup_hour,dropoff_year,dropoff_month,dropoff_dayofweek,dropoff_hour,pickupDayofWeek_TR,dropoffDayofWeek_TR,dropoffMonth_TR,pickupMonth_TR
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,...,2,17,2016,3,2,17,Pazartesi,Pazartesi,Mart,Mart
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,...,1,0,2016,6,1,0,Pazar,Pazar,Haziran,Haziran
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,...,3,11,2016,1,3,12,Salı,Salı,Ocak,Ocak
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,...,4,19,2016,4,4,19,Çarşamba,Çarşamba,Nisan,Nisan
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,...,7,13,2016,3,7,13,Cumartesi,Cumartesi,Mart,Mart


In [22]:
df9 = df8.withColumn("id",
            F.regexp_replace(F.col("id"), "[id]", ""))
df9.limit(5).toPandas()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,pickup_dayofweek,pickup_hour,dropoff_year,dropoff_month,dropoff_dayofweek,dropoff_hour,pickupDayofWeek_TR,dropoffDayofWeek_TR,dropoffMonth_TR,pickupMonth_TR
0,2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,...,2,17,2016,3,2,17,Pazartesi,Pazartesi,Mart,Mart
1,2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,...,1,0,2016,6,1,0,Pazar,Pazar,Haziran,Haziran
2,3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,...,3,11,2016,1,3,12,Salı,Salı,Ocak,Ocak
3,3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,...,4,19,2016,4,4,19,Çarşamba,Çarşamba,Nisan,Nisan
4,2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,...,7,13,2016,3,7,13,Cumartesi,Cumartesi,Mart,Mart


In [23]:
df9.limit(5).toPandas()

Traceback (most recent call last):                                  (0 + 1) / 1]
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,pickup_dayofweek,pickup_hour,dropoff_year,dropoff_month,dropoff_dayofweek,dropoff_hour,pickupDayofWeek_TR,dropoffDayofWeek_TR,dropoffMonth_TR,pickupMonth_TR
0,2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,...,2,17,2016,3,2,17,Pazartesi,Pazartesi,Mart,Mart
1,2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,...,1,0,2016,6,1,0,Pazar,Pazar,Haziran,Haziran
2,3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,...,3,11,2016,1,3,12,Salı,Salı,Ocak,Ocak
3,3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,...,4,19,2016,4,4,19,Çarşamba,Çarşamba,Nisan,Nisan
4,2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,...,7,13,2016,3,7,13,Cumartesi,Cumartesi,Mart,Mart


In [24]:
from math import radians, cos, sin, asin, sqrt
from pyspark.sql.types import FloatType

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [26]:
haversine_distance = F.udf(lambda lon1, lat1, lon2, lat2: haversine(lon1, lat1, lon2, lat2), FloatType())
spark.udf.register("haversine_distance", haversine_distance)

<function __main__.<lambda>(lon1, lat1, lon2, lat2)>

In [31]:
df10 = df9.withColumn("haversine_distance(km)", 
        haversine_distance(F.col("pickup_longitude"), F.col("pickup_latitude"), F.col("dropoff_longitude"), F.col("dropoff_latitude")))

In [32]:
df10.limit(5).toPandas()

Traceback (most recent call last):
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,pickup_hour,dropoff_year,dropoff_month,dropoff_dayofweek,dropoff_hour,pickupDayofWeek_TR,dropoffDayofWeek_TR,dropoffMonth_TR,pickupMonth_TR,haversine_distance(km)
0,2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,...,17,2016,3,2,17,Pazartesi,Pazartesi,Mart,Mart,1.498521
1,2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,...,0,2016,6,1,0,Pazar,Pazar,Haziran,Haziran,1.805507
2,3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,...,11,2016,1,3,12,Salı,Salı,Ocak,Ocak,6.385098
3,3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,...,19,2016,4,4,19,Çarşamba,Çarşamba,Nisan,Nisan,1.485498
4,2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,...,13,2016,3,7,13,Cumartesi,Cumartesi,Mart,Mart,1.188589


In [22]:
spark.stop()